In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from tensorflow import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
# from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
from keras.optimizers import *
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

F:\anaconda\envs\kimevn\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\anaconda\envs\kimevn\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\anaconda\envs\kimevn\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\anaconda\envs\kimevn\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth= True
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))# sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
np.random.seed(3407)  
tf.set_random_seed(3407)

KTF.set_session(session)

In [4]:
from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *

In [5]:
data_root_path = 'MINDsmall\\'
embedding_path = 'glove.840B.300d\\'
KG_root_path = 'KGData\\'

In [6]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(data_root_path)
news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)


preprocessing-read_news
path: MINDsmall\train\news.tsv
path: MINDsmall\dev\news.tsv


In [7]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path, word_dict)
content_word_embedding_matrix, have_word = load_matrix(embedding_path, content_dict)

load_matrix:
load_matrix:


In [8]:
graph, EntityId2Index, EntityIndex2Id, entity_embedding = load_entity_metadata(KG_root_path)
news_entity_KG = load_news_entity(news,EntityId2Index,data_root_path)


load_entity_metadata
load_news_entity


In [9]:
news_entity_index = parse_zero_hop_entity(EntityId2Index,news_entity_KG,news_index,max_entity_num)
one_hop_entity = parse_one_hop_entity(EntityId2Index,EntityIndex2Id,news_entity_index,graph,news_index,max_entity_num)

In [10]:
train_session = read_train_clickhistory(news_index, data_root_path, 'train/behaviors.tsv')
train_user = parse_user(news_index, train_session)
train_sess, train_user_id, train_label = get_train_input(news_index, train_session)

read_train_clickhistory()
get_train_input()


In [11]:
news_fetcher = NewsFetcher(news_title, news_content, news_vert, news_subvert, news_entity)  # 传递，设为共有
test_session = read_test_clickhistory_noclk(news_index, data_root_path, 'dev/behaviors.tsv')

In [12]:
test_user = parse_user(news_index, test_session)
test_docids, test_userids, test_labels, test_bound = get_test_input(news_index, test_session)

In [13]:
train_generator = get_hir_train_generator(news_fetcher, news_entity_index, one_hop_entity, entity_embedding,
                                          train_user['click'], train_user_id, train_sess, train_label, 16)

test_generator = get_test_generator(test_docids,test_userids,news_fetcher,news_entity_index,one_hop_entity,entity_embedding,test_user['click'],64)


In [14]:
model, inter_model = create_model_new(title_word_embedding_matrix, content_word_embedding_matrix, entity_dict, category_dict, subcategory_dict)

create_model_new
1
2
3
4
5
6
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
candi_news_words: Tensor("dropout_12/cond/Merge:0", shape=(?, 400), dtype=float32)
user_vecs: Tensor("dropout_21/cond/Merge:0", shape=(?, 1600, 300), dtype=float32)
user_vecs: Tensor("fastformer_1/Reshape_21:0", shape=(?, 1600, 400), dtype=float32)
user_vecs: Tensor("dropout_23/cond/Merge:0", shape=(?, 1600, 400), dtype=float32)
user_vecs: Tensor("reshape_20/Reshape:0", shape=(?, 50, 32, 400), dtype=float32)
7
8
9
10
11
12
13


In [ ]:
model.fit_generator(train_generator, epochs=5, verbose=1)

predicted_label = inter_model.predict_generator(test_generator, verbose=1)

AUC, MRR, nDCG5, nDCG10 = evaluate(predicted_label, test_labels, test_bound)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
